In [ ]:
pip install torch==1.11.0+cu102 torchvision==0.12.0+cu102 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu102

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu102
     |████████████████████████████████| 750.6 MB 15 kB/s 
     |████████████████████████████████| 21.0 MB 1.2 MB/s 
     |████████████████████████████████| 2.9 MB 40.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0+cu116
    Uninstalling torchvision-0.14.0+cu116:
      Successfully uninstalled torchvision-0.14.0+cu116
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.13.0+cu116
    Uninstalling torchaudio-0.13.0+cu116:
      Successfully uninstalled torchaudio-0.13.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving MDEA.csv to MDEA.csv


In [ ]:
import pandas as pd
import io
 
df = pd.read_csv(io.BytesIO(uploaded['MDEA.csv']))
print(df)

     Acid gas  HSAS  Velocity  Temperature  Corrosion Rate
0        0.15  0.50       1.5           88            0.03
1        0.15  2.25       1.5           88            0.05
2        0.15  4.00       1.5           88            0.13
3        0.25  0.50       1.5           88            0.05
4        0.25  2.25       1.5           88            0.10
..        ...   ...       ...          ...             ...
751      0.55  4.00       3.0          127            3.05
752      0.45  2.25       1.5           88            0.51
753      0.45  0.50       3.0          127            3.81
754      0.55  4.00       3.0          127            5.08
755      0.55  2.25       3.0          127            1.91

[756 rows x 5 columns]


In [ ]:
df_array = df.to_numpy()

In [ ]:
df_array_input = df_array[:,:4]

In [ ]:
df_array_input

array([[  0.15,   0.5 ,   1.5 ,  88.  ],
       [  0.15,   2.25,   1.5 ,  88.  ],
       [  0.15,   4.  ,   1.5 ,  88.  ],
       ...,
       [  0.45,   0.5 ,   3.  , 127.  ],
       [  0.55,   4.  ,   3.  , 127.  ],
       [  0.55,   2.25,   3.  , 127.  ]])

In [ ]:
!pip install GPy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 959 kB 7.0 MB/s 
     |████████████████████████████████| 71 kB 4.9 MB/s 
  Created wheel for GPy: filename=GPy-1.10.0-cp38-cp38-linux_x86_64.whl size=2783587 sha256=bd451ee6f1c9fed80684b8cf46ea519fdf997b4a5ae1f325e23ed2c8529728c2
  Stored in directory: /root/.cache/pip/wheels/48/b3/22/31f07cfd7b182ea3703151b7e5a7d6447e3e1ac6aa5c529413
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102566 sha256=5828e975abd95f7ab4cf756ef38f05cc03e7fae6241fa48c804252eee3b49050
  Stored in directory: /root/.cache/pip/wheels/66/78/6c/d98cb437834de5e29381786b4ba8a77ea68cca74653ab62713
Successfully built GPy paramz


In [ ]:
import GPy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! git clone https://github.com/pmorenoz/ModularGP.git

Cloning into 'ModularGP'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 100 (delta 34), reused 66 (delta 20), pack-reused 0
Receiving objects: 100% (100/100), 788.92 KiB | 14.09 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [ ]:
cd /content/ModularGP/

/content/ModularGP


In [ ]:
%cd experiments

/content/ModularGP/experiments


In [ ]:
x = df_array_input
x = np.log(x + 1)
x = x[:,np.newaxis]
x = (x - np.mean(x))
x = x.reshape(756,4)   ##### mean normalization

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2, svd_solver='full')
pca_scores = pca.fit_transform(x)
x = pca_scores[:,0]

In [ ]:
x = x.reshape(-1,1)

In [ ]:
from kernels.rbf import RBF
from likelihoods.gaussian import Gaussian
from models.svgp import SVGP
from models.ensemblegp import EnsembleGP
from baselines.distgp import DistGP
from baselines.poegp import PoeGP
from baselines.gpoegp import GenPoeGP
from baselines.bcm import BayesianCM
from baselines.rbcm import RobustBayesianCM
from baselines.dvigp import DVIGP
from optimization.algorithms import AlgorithmVEM
from optimization.algorithms import GPR_Optimizer
from optimization.algorithms import AlgorithmVEM
from sklearn.model_selection import train_test_split

import torch
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

# COOLORS.CO palettes
color_palette_1 = ['#335c67','#fff3b0','#e09f3e','#9e2a2b','#540b0e']
color_palette_2 = ['#177e89','#084c61','#db3a34','#ef8354','#323031']
color_palette_3 = ['#bce784','#5dd39e','#348aa7','#525274','#513b56']
color_palette_4 = ['#002642','#840032','#e59500','#e5dada','#02040e']
color_palette_5 = ['#202c39','#283845','#b8b08d','#f2d449','#f29559']
color_palette_6 = ['#21295c','#1b3b6f','#065a82','#1c7293','#9eb3c2']
color_palette_7 = ['#f7b267','#f79d65','#f4845f','#f27059','#f25c54']

palette = color_palette_4

trials = 2
experiment = 'MDEA'

recy_metrics = np.zeros((3,trials))
poe_metrics = np.zeros((3,trials))
gpoe_metrics = np.zeros((3,trials))
bcm_metrics = np.zeros((3,trials))
rbcm_metrics = np.zeros((3,trials))

# Load Solar Data --
#data = sio.loadmat('/content/nasa.mat')
y = df_array[:,4]
#y = y.reshape(-1,1)
y = np.log(y + 1)
y = y[:,np.newaxis]
y = (y - np.mean(y))    # mean normalization


print(y.shape)
print(x.shape)

for trial in range(trials):

    print('TRIAL = ' + str(trial) + '/' + str(trials))

    ###########################
    #                         #
    #    DISTRIBUTED TASKS    #
    #                         #
    ###########################

    tasks = 3
    min_x = x.min()
    max_x = x.max()
    segment_x = (max_x - min_x)/tasks
    x_tasks = []    # training x -- inputs
    y_tasks = []    # training y -- outputs

    x_test = torch.zeros(1,1)     # test x -- inputs
    y_test = torch.zeros(1,1)     # test y -- outputs

    n_training = 0
    n_test = 0
    for k in range(tasks):
        min_x_k = min_x + (k*segment_x)
        max_x_k = min_x + ((k+1)*segment_x)
        y_k = y[(x[:, 0] > min_x_k) & (x[:, 0] < max_x_k), :]
        x_k = x[(x[:, 0] > min_x_k) & (x[:, 0] < max_x_k), :]
        # y_k_1 = y[(x[:,1]>min_x_k) & (x[:,1]<max_x_k),:]
        # x_k_1 = x[(x[:,1]>min_x_k) & (x[:,1]<max_x_k),:]
        # y_k_2 = y[(x[:,2]>min_x_k) & (x[:,2]<max_x_k),:]
        # x_k_2 = x[(x[:,2]>min_x_k) & (x[:,2]<max_x_k),:]
        # y_k_3 = y[(x[:,3]>min_x_k) & (x[:,3]<max_x_k),:]
        # x_k_3 = x[(x[:,3]>min_x_k) & (x[:,3]<max_x_k),:] 
        # y_k = np.concatenate((y_k_0, y_k_1, y_k_2, y_k_3))
        # x_k = np.concatenate((x_k_0, x_k_1, x_k_2, x_k_3))

        x_k_train, x_k_test, y_k_train, y_k_test = train_test_split(x_k, y_k, test_size = 0.2, random_state = 42)

        x_tasks.append(torch.from_numpy(x_k_train).float())
        y_tasks.append(torch.from_numpy(y_k_train).float())

        x_test = torch.cat((x_test, torch.from_numpy(x_k_test).float()), 0)
        y_test = torch.cat((y_test, torch.from_numpy(y_k_test).float()), 0)


        #x_k_test = x_k[::5, :]
        #y_k_test = y_k[::5, :]

        #x_tasks.append(torch.from_numpy(np.delete(x_k,np.s_[::5])[:,None]).float())
        #y_tasks.append(torch.from_numpy(np.delete(y_k,np.s_[::5])[:,None]).float())

        #x_test = torch.cat((x_test, torch.from_numpy(x_k_test).float()), 0)
        #y_test = torch.cat((y_test, torch.from_numpy(y_k_test).float()), 0)

        n_training += y_k_train.shape[0]
        n_test += y_k_test.shape[0]


    print('Total # of tasks: ', len(x_tasks))
    print('Number # of training samples: ', n_training)
    print('Number # of test samples: ', n_test)

    ###########################
    #                         #
    #   PARALLEL INFERENCE    #
    #                         #
    ###########################

    M_k = 3                                                                     ###### inducing variables
    models = []       # for recyclable GPs
    models_dist = []  # for distributed GPs
    x_all = []        # for distributed GPs
    y_all = []        # for distributed GPs
    for k, x_k in enumerate(x_tasks):
        print('-                             -')
        print('----- TASK k=' + str(k + 1) + ' ------')
        print('-                             -')
        ######################################################
        # 1. RECYCLABLE GP
        ######################################################
        kernel_k = RBF(length_scale=0.2, variance=1.0)                          ##### kernel
        likelihood_k = Gaussian(sigma=0.1, fit_noise=True)
        model_k = SVGP(kernel_k, likelihood_k, M_k)                             ##### finds the posterior distribution taking into account the prior and likelihood

        z_k_min = min_x + (k*segment_x)
        z_k_max = min_x + ((k+1)*segment_x)
        model_k.z = torch.nn.Parameter(torch.linspace(z_k_min, z_k_max, M_k)[:,None], requires_grad=True)  #### Registers the inducing points as model parameter
                                                                                ### To find optimal values for them, parameter learning uses gradient descent to maximize the objective function log(p(y)) with respect to those model parameters.
        
        vem_algorithm = AlgorithmVEM(model_k, x_k, y_tasks[k], iters=20)        #### The Vem algorithm figures out where the inducing locations should be

        vem_algorithm.ve_its = 3
        vem_algorithm.vm_its = 3
        vem_algorithm.lr_m = 1e-5
        vem_algorithm.lr_L = 1e-8
        vem_algorithm.lr_hyp = 1e-10
        vem_algorithm.lr_z = 1e-10

        vem_algorithm.fit()

        ######################################################
        # 2. DISTRIBUTED GP (FOR BCM, RBCM, POE & GPOE)
        ######################################################

        kernel_j = RBF()
        likelihood_j = Gaussian(fit_noise=False)
        model_j = DistGP(kernel_j, likelihood_j)
        GPR_Optimizer(model_j, x_k, y_tasks[k])

        models_dist.append(model_j)    ############## optimized model is appended in a list. (list contains optimized GP models or meta models)
        x_all.append(x_k)              ############## contains a dictionary of already fitted GP modules for later use
        y_all.append(y_tasks[k])

    ###########################
    #                         #
    #   ENSEMBLE INFERENCE    #
    #                         #
    ###########################
    print('-                   -')
    print('----- ENSEMBLE ------')
    print('-                   -')

    ######################################################
    # 1. RECYCLABLE GP
    ######################################################

    M_e = 5
    kernel = RBF()
    likelihood = Gaussian(fit_noise=False)
    model_e = EnsembleGP(kernel, likelihood, models, M_e)
    model_e.z = torch.nn.Parameter(torch.linspace(min_x, max_x, M_e)[:,None], requires_grad=True)
    vem_algorithm = AlgorithmVEM(model_e, config='ensemble', iters=10)

    vem_algorithm.ve_its = 5
    vem_algorithm.vm_its = 5
    vem_algorithm.lr_m = 1e-3
    vem_algorithm.lr_L = 1e-6
    vem_algorithm.lr_hyp = 1e-8
    vem_algorithm.lr_z = 1e-8

    vem_algorithm.fit()

    nlpd = model_e.nlpd(x_test, y_test)
    rmse = model_e.rmse(x_test, y_test)
    mae = model_e.mae(x_test, y_test)

    recy_metrics[0, trial] = nlpd
    recy_metrics[1, trial] = rmse
    recy_metrics[2, trial] = mae

    print('Recyclable - NLPD: ', nlpd)
    print('Recyclable - RMSE: ', rmse)
    print('Recyclable - MAE: ', mae)
    print(' ')

    ######################################################
    # 2. DISTRIBUTED GP (FOR BCM, RBCM, POE & GPOE)
    ######################################################

    # A. POE  _________//

    poe_model = PoeGP(models_dist)                                              ##### takes in a list of modules and every Dist GP model has its own hyperparamters to tune

    nlpd = poe_model.nlpd(x_all, y_all, x_test, y_test)                         ##### x_all consists of 3 tensors and x_test consisy of 1 tensor. Similar to GLM, whichever tensor
    rmse = poe_model.rmse(x_all, y_all, x_test, y_test)                         ##### in x_all is close to x_test, that tensor is compared.
    mae = poe_model.mae(x_all, y_all, x_test, y_test)

    poe_metrics[0, trial] = nlpd
    poe_metrics[1, trial] = rmse
    poe_metrics[2, trial] = mae

    print('POE-NLPD: ', nlpd)
    print('POE-RMSE: ', rmse)
    print('POE-MAE: ',  mae)
    print(' ')

    # B. GPOE _________//

    gpoe_model = GenPoeGP(models_dist)

    nlpd = gpoe_model.nlpd(x_all, y_all, x_test, y_test)
    rmse = gpoe_model.rmse(x_all, y_all, x_test, y_test)
    mae = gpoe_model.mae(x_all, y_all, x_test, y_test)

    gpoe_metrics[0, trial] = nlpd
    gpoe_metrics[1, trial] = rmse
    gpoe_metrics[2, trial] = mae

    print('GenPOE-NLPD: ', nlpd)
    print('GenPOE-RMSE: ', rmse)
    print('GenPOE-MAE: ',  mae)
    print(' ')

    # # C. BCM  _________//

    # bcm_model = BayesianCM(models_dist)

    # nlpd = bcm_model.nlpd(x_all, y_all, x_test, y_test)
    # rmse = bcm_model.rmse(x_all, y_all, x_test, y_test)
    # mae = bcm_model.mae(x_all, y_all, x_test, y_test)

    # bcm_metrics[0, trial] = nlpd
    # bcm_metrics[1, trial] = rmse
    # bcm_metrics[2, trial] = mae

    # print('BCM-NLPD: ', nlpd)
    # print('BCM-RMSE: ', rmse)
    # print('BCM-MAE: ',  mae)
    # print(' ')

    # D. RBCM _________//

    rbcm_model = RobustBayesianCM(models_dist)

    nlpd = rbcm_model.nlpd(x_all, y_all, x_test, y_test)
    rmse = rbcm_model.rmse(x_all, y_all, x_test, y_test)
    mae = rbcm_model.mae(x_all, y_all, x_test, y_test)

    rbcm_metrics[0, trial] = nlpd
    rbcm_metrics[1, trial] = rmse
    rbcm_metrics[2, trial] = mae

    print('RBCM-NLPD: ', nlpd)
    print('RBCM-RMSE: ', rmse)
    print('RBCM-MAE: ',  mae)
    print(' ')

    # save to csv file
    np.savetxt('MDEA_recy_metrics.csv', recy_metrics, delimiter=',')
    np.savetxt('MDEA_poe_metrics.csv', poe_metrics, delimiter=',')
    np.savetxt('MDEA_gpoe_metrics.csv', gpoe_metrics, delimiter=',')
    #np.savetxt('./metrics/bcm_metrics_' + experiment + '.csv', bcm_metrics, delimiter=',')
    np.savetxt('MDEA_rbcm_metrics.csv', rbcm_metrics, delimiter=',')

(756, 1)
(756, 1)
TRIAL = 0/2
Total # of tasks:  3
Number # of training samples:  599
Number # of test samples:  152
-                             -
----- TASK k=1 ------
-                             -
Variational EM step (it=0)


 /content/ModularGP/kernels/rbf.py:31: UserWarning:torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).mH().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1337.)
 /content/ModularGP/models/svgp.py:74: UserWarning:torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ../aten/src/ATen/native/BatchLin

  \__ elbo = 723034048.0
Variational EM step (it=1)
  \__ elbo = 8739875840.0
Variational EM step (it=2)
  \__ elbo = 20199096320.0
Variational EM step (it=3)
  \__ elbo = 31536107520.0
Variational EM step (it=4)
  \__ elbo = 41385250816.0
Variational EM step (it=5)
  \__ elbo = 49406300160.0
Variational EM step (it=6)
  \__ elbo = 55699501056.0
Variational EM step (it=7)
  \__ elbo = 60523094016.0
Variational EM step (it=8)
  \__ elbo = 64163725312.0
Variational EM step (it=9)
  \__ elbo = 66882613248.0
Variational EM step (it=10)
  \__ elbo = 68898062336.0
Variational EM step (it=11)
  \__ elbo = 70383894528.0
Variational EM step (it=12)
  \__ elbo = 71474765824.0
Variational EM step (it=13)
  \__ elbo = 72272961536.0
Variational EM step (it=14)
  \__ elbo = 72855273472.0
Variational EM step (it=15)
  \__ elbo = 73278881792.0
Variational EM step (it=16)
  \__ elbo = 73586089984.0
Variational EM step (it=17)
  \__ elbo = 73808035840.0
Variational EM step (it=18)
  \__ elbo = 739676569